<a href="https://colab.research.google.com/github/BillWENZE/BillWENZE/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
! pip install yfinance

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4143    0  4143    0     0   4151      0 --:--:-- --:--:-- --:--:--  4151
100  517k  100  517k    0     0   201k      0  0:00:02  0:00:02 --:--:--  683k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4155    0  4155    0     0   7949      0 --:--:-- --:--:-- --:--:--  7959
100  392k  100  392k    0     0   175k      0  0:00:02  0:00:02 --:--:--  293k


In [3]:
pip install scikit-learn

In [ ]:
pip install TA-Lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import talib as ta
from talib import MA_Type
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import TimeSeriesSplit as tscv

In [ ]:
msft=yf.Ticker('MSFT')
df=msft.history('max')
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.054693,0.062736,0.054693,0.060055,1031788800,0.00,0.0
1986-03-14 00:00:00-05:00,0.060055,0.063272,0.060055,0.062199,308160000,0.00,0.0
1986-03-17 00:00:00-05:00,0.062199,0.063808,0.062199,0.063272,133171200,0.00,0.0
1986-03-18 00:00:00-05:00,0.063272,0.063808,0.061127,0.061663,67766400,0.00,0.0
1986-03-19 00:00:00-05:00,0.061663,0.062199,0.060055,0.060591,47894400,0.00,0.0
...,...,...,...,...,...,...,...
2024-05-10 00:00:00-04:00,412.196509,414.632118,411.058547,413.993256,13402300,0.00,0.0
2024-05-13 00:00:00-04:00,417.257382,417.596766,410.080325,412.975098,15440200,0.00,0.0
2024-05-14 00:00:00-04:00,411.278163,416.738316,410.809008,415.809998,15109300,0.00,0.0


In [ ]:
help(ta.CCI)

Help on function CCI in module talib._ta_lib:

CCI(high, low, close, timeperiod=-2147483648)
    CCI(high, low, close[, timeperiod=?])
    
    Commodity Channel Index (Momentum Indicators)
    
    Inputs:
        prices: ['high', 'low', 'close']
    Parameters:
        timeperiod: 14
    Outputs:
        real



In [ ]:
df['High Shifted']=df['High'].shift(1)
df['Low Shifted'] = df['Low'].shift(1)
df['Close Shifted'] = df['Close'].shift(1)
df['Upper BBand'], df['Middle BBand'],df['Lower BBand']= ta.BBANDS(df['Close Shifted'],timeperiod=20)
df['RSI'] = ta.RSI(np.array(df['Close Shifted']), timeperiod=14)
df['Macd'], df['Macd Signal'],df['Macd Hist'] = ta.MACD(df['Close Shifted'], fastperiod=12, slowperiod=26, signalperiod=9)
df['Momentum'] = ta.MOM(df['Close Shifted'],timeperiod=12)
df['OBV'] = ta.OBV(df['Close Shifted'],df['Volume'].shift(1))
df['ATR'] = ta.ATR(df['High'].shift(1),df['Low'].shift(1),df['Close'].shift(1))
df['Returns'] = np.log(df['Close']/df['Close'].shift(1))
df['CCI']=ta.CCI(df['High Shifted'],df['Low Shifted'],df['Close Shifted'])
df.dropna(inplace=True)

In [ ]:
def label_returns(r):
    if r >= 0.01:
        return 1  # Long the stock
    elif r <= -0.01:
        return -1  # Short the stock
    else:
        return 0  # Do nothing

# Apply the function to create the label
df['Label'] = df['Returns'].apply(label_returns)
df['Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df['Returns']

Date
1986-05-01 00:00:00-04:00         NaN
1986-05-02 00:00:00-04:00    0.000000
1986-05-05 00:00:00-04:00   -0.007905
1986-05-06 00:00:00-04:00    0.007905
1986-05-07 00:00:00-04:00    0.000000
                               ...   
2024-05-10 00:00:00-04:00    0.005852
2024-05-13 00:00:00-04:00   -0.002462
2024-05-14 00:00:00-04:00    0.006841
2024-05-15 00:00:00-04:00    0.017333
2024-05-16 00:00:00-04:00   -0.004952
Name: Returns, Length: 9588, dtype: float64

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Exclude specified columns to create the feature space X and the labels y

X = df.drop(columns=['Label', 'Returns', 'Open', 'Close', 'Volume', 'High', 'Low', 'Dividends', 'Stock Splits'])

In [ ]:
y = df['Label']  # Assuming 'Label' contains your label data
# Split the dataset: All data except the last 30 days for training, last 30 days for testing
train_data = df.iloc[:-30]
test_data = df.iloc[-30:]

# Extract features and labels for both training and testing sets
X_train = train_data.drop(columns=['Label', 'Returns', 'Open', 'Close', 'Volume', 'High', 'Low', 'Dividends', 'Stock Splits'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['Label', 'Returns', 'Open', 'Close', 'Volume', 'High', 'Low', 'Dividends', 'Stock Splits'])
y_test = test_data['Label']

# Initialize and apply MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Sample data loading and preparation
# X, y = load_your_data()

# Time Series Split configuration
tscv = TimeSeriesSplit(n_splits=5, test_size=1, gap=0)

# Parameter grid for RandomizedSearchCV
param_distributions = {
    'C': [0.3, 0.5, 1, 5, 10, 20, 30, 50, 100],
    'kernel': ['linear', 'rbf']
}

# SVM model initialization
svm = SVC()

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=10,  # Number of parameter settings sampled (adjust as needed)
    scoring='f1_macro',
    cv=tscv,
    verbose=1,
    random_state=42
)

# Fitting RandomizedSearchCV
random_search.fit(X, y)

# Best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_

# Assuming you have a separate test set
# X_test, y_test = load_your_test_data()
y_test_pred = random_search.predict(X_test)

# Evaluating performance on the test data
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='macro')
test_recall = recall_score(y_test, y_test_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

print(f"Best Parameters: {best_params}")
print(f"Best CV Score: {best_score}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score


In [ ]:
# Setting up the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5, test_size=1, gap=0)

# Parameters for GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 0.5, 1, 2, 3, 4, 5, 10]
}

# Setting up the Logistic Regression model
model = LogisticRegression(solver='liblinear')

# Setting up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='accuracy')

# Fitting GridSearchCV
grid_search.fit(X, y)

In [ ]:

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Evaluating on the training set (using the best estimator)
y_train_pred = grid_search.predict(X)
train_accuracy = accuracy_score(y, y_train_pred)

# Assuming you have a separate test set
# X_test, y_test = load_your_test_data()
y_test_pred = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Best Parameters: {best_params}")
print(f"Best CV Score: {best_score}")
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Sample data preparation
# X, y = load_your_data()  # Load your feature matrix X and label vector y

# Time Series Cross-Validator
tscv = TimeSeriesSplit(n_splits=5, test_size=1, gap=0)

# Parameter distribution for RandomizedSearchCV
param_distributions = {
    'C': [0.3, 0.5, 1, 5, 10, 20, 30, 50, 100],
    'kernel': ['linear', 'rbf']
}

# SVM model initialization
svm_model = SVC()

# Randomized Search with Cross-Validation
random_search = RandomizedSearchCV(svm_model, param_distributions=param_distributions, n_iter=10,
                                   scoring='f1_macro', cv=tscv, verbose=2, random_state=42)
random_search.fit(X, y)

# Best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_

# Evaluate on the training set
y_train_pred = random_search.predict(X)
train_accuracy = accuracy_score(y, y_train_pred)

# Assuming you have a separate test set
# X_test, y_test = load_your_test_data()  # Load your test data
y_test_pred = random_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='macro')
test_recall = recall_score(y_test, y_test_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

print(f"Best Parameters: {best_params}")
print(f"Best CV Score: {best_score}")
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")
